In [1]:
import sys
sys.path.append("..")
import os
os.environ['USE_PYGEOS'] = '0'
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols 
from rasterstats import zonal_stats
import rasterio
import contextily as cx
from rasterio.plot import show
#import osmnx as ox
from affine import Affine
import rioxarray as rx
from multiprocessing import Pool, cpu_count
from shapely.ops import unary_union
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
from pathlib import Path
import fiona

#see all columns in tables
pd.set_option('display.max_columns', None)


%load_ext autoreload
%autoreload 2

In [16]:
from src.features.parcel_model import parcel_ms4_model

Reading in municipal and regional boundaries...
Reading in land parcel database data...
Reading in additional data layers...
Wetlands...
Watersheds...
Wellhead protection areas...
Activity use limitation areas...
ParkServe data...


In [62]:
from src.data.make_dataset import munis

processed_path = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Output\\Parcels"

for town_name in munis['municipal'].unique():
    parcel_ms4_model(town_name, processed_path)

'''
#run suitability model for each town in list
for town_name in munis['municipal'].unique():
    if town_name in os.listdir(processed_path): #currently set to skip any towns for which data has already been produced
        pass
    else:
        parcel_ms4_model(town_name, processed_path)
'''

Starting on Lexington at 2023-07-03 19:44:52...


KeyError: 'muni_id'

In [24]:
# MERGE SHAPEFILES 
from src.data.make_dataset import munis

model_path = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Model"

filelist = []

# add all shapefiles - takes about 30 minutes
for root, folder, files in os.walk(processed_path):
    for file in files:
        for muni in munis['municipal'].unique():
            if muni in file:
                if file.endswith('.shp'):
                    fullname = os.path.join(root, file)
                    filelist.append(fullname)


merged_gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in filelist], 
                        ignore_index=True), crs=gpd.read_file(filelist[0]).crs)

#export
merged_gdf.to_file(model_path + '\\Merged_MS4_Parcels.shp')


In [2]:
from src.data.make_dataset import munis
munis.head()

Reading in municipal and regional boundaries...
Reading in land parcel database data...
Reading in additional data layers...
Wetlands...
Watersheds...
Wellhead protection areas...
Activity use limitation areas...
ParkServe data...


,muni_id,municipal,geometry
0,155,Lexington,"POLYGON ((223675.913 912896.955, 223681.822 91..."
1,93,Everett,"MULTIPOLYGON (((238796.142 908223.795, 238627...."
2,25,Bellingham,"POLYGON ((204842.472 876046.525, 204820.983 87..."
3,187,Millis,"POLYGON ((213252.583 877673.165, 213242.623 87..."
4,92,Essex,"MULTIPOLYGON (((259231.583 936048.315, 259275...."


In [36]:
from src.features.parcel_model import parcel_ms4_model

#rerun for Boston and then replace Boston parcels with the new ones?
processed_path = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Output\\Parcels"
boston_sites = parcel_ms4_model('Boston', processed_path)

from src.data.make_dataset import munis

model_path = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Model"

filelist = []

# add all shapefiles - takes about 30 minutes
for root, folder, files in os.walk(processed_path):
    for file in files:
        for muni in munis['municipal'].unique():
            if muni in file:
                if file.endswith('.shp'):
                    fullname = os.path.join(root, file)
                    filelist.append(fullname)


merged_gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in filelist], 
                        ignore_index=True), crs=gpd.read_file(filelist[0]).crs)

#export
merged_gdf.to_file(model_path + '\\Merged_MS4_Parcels.shp')


Starting on Boston at 2023-07-04 11:27:08...
getting row data from Boston
<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 7783 entries, 0 to 7782
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   parloc_id               7783 non-null   int64   
 1   FID_Bos_InterBuff_Clip  7783 non-null   int64   
 2   FID_Bos_ROWparcels      7783 non-null   int64   
 3   ORIG_FID                7783 non-null   int64   
 4   Shape_Length            7783 non-null   float64 
 5   Shape_Area              7783 non-null   float64 
 6   geometry                7783 non-null   geometry
dtypes: float64(2), geometry(1), int64(4)
memory usage: 486.4 KB
None
<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 7783 entries, 0 to 7782
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   parloc_id  7783 non-null   object  
 1   geometry   778

### Identify muni/munis of interest, create a shapefile for the muni

Then create land use and land cover layers of interest based on the municipal boundary mask (speeds up processes)

### Pull base geographies for analysis
- Get municipal ROW and fishnet (tesselation) layers from geodatabases 
- Clean up those gdfs to include only the relevant info

`town_parcels_row` and `town_tess` are the two base geographies for this analysis.